In [1]:
import speech_recognition as sr
import pyaudio
import os
from openai import OpenAI
from gtts import gTTS
import pygame
import gradio as gr

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
pygame 2.6.1 (SDL 2.28.4, Python 3.11.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


/Users/parakhchaudhary/opt/anaconda3/envs/chaenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Speech Recognition

In [2]:

recognizer = sr.Recognizer()

def speech_to_text():
    with sr.Microphone() as source:
        print("Adjusting for ambient noise...")
        recognizer.adjust_for_ambient_noise(source, duration=1) 
        print("Please speak...")
        try:
            audio_data = recognizer.listen(source, timeout=5) 
            print("Recognizing with PocketSphinx...")
            text = recognizer.recognize_sphinx(audio_data)
            print("Recognized Text: " + text)
        except sr.UnknownValueError:
            print("Could not understand the audio.")
        except Exception as e:
            print(f"Error: {e}")
        return text


LLM

In [3]:
def text_to_meaning(input):
    monster_api_key = os.getenv('MONSTER_API_KEY')

    generation_model_name: str
    temperature: float = 0.9
    top_p = 0.9
    max_tokens: int = 2048
    stream: bool = False  
    llm_name: str = "Meta-Llama"

    monster_client = OpenAI(
        base_url="https://llm.monsterapi.ai/v1/",
        api_key=monster_api_key
    )

    monster_ai_model_name = {
        "Google-Gemma": "google/gemma-2-9b-it",
        "Mistral": "mistralai/Mistral-7B-Instruct-v0.2",
        "Microsoft-Phi": "microsoft/Phi-3-mini-4k-instruct",
        "Meta-Llama": "meta-llama/Meta-Llama-3.1-8B-Instruct",
    }
    message = [
                {"role": "system", "content": "You are an AI assistant. Kindly answer the following question:"},
                {"role": "user", "content": input}
            ]

    response = monster_client.chat.completions.create( model=monster_ai_model_name[llm_name], messages=message, temperature=temperature, top_p=top_p, max_tokens=max_tokens, stream=False)
    return (response.choices[0].message.content)

Text to Speech

In [4]:
def speak_output(output):
    language = "en"
    myobj = gTTS(text=output,lang = language,slow = False)
    myobj.save("welcome.mp3")
    pygame.mixer.init()
    pygame.mixer.music.load("welcome.mp3")
    pygame.mixer.music.play()

Main Function

In [ ]:
# orchestrator

with gr.Blocks(theme=gr.themes.Glass()) as interface:
    gr.Markdown("## Speech 2 Speech LLM Application")
    gr.Markdown("_Developed for Tensorgo_")
    
    with gr.Tab("🎤 Ask Your Question"):
        input_box = gr.Textbox(label="Input Text", placeholder="Speak or type your query here...")
        output_text_box = gr.Textbox(label="Output Text")
        output_audio = gr.Audio(label="TTS Output", type="filepath",autoplay=True)
        mic_button = gr.Button("🎙️ Speak Now")
        process_button = gr.Button("🔍 Process Query")
        def process_query(input_text):
            # Check if input_text is empty
            if not input_text.strip():
                return "Please provide a query.", None
            
            # Process the input text using LLM
            output_text = text_to_meaning(input_text)
            
            # Generate TTS output
            language = "en"
            tts = gTTS(text=output_text, lang=language, slow=False)
            tts_file = "output.mp3"
            tts.save(tts_file)
            
            return output_text, tts_file
        process_button.click(process_query, inputs=input_box, outputs=[output_text_box, output_audio])
        mic_button.click(speech_to_text, outputs=input_box)

    
    

interface.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


IMPORTANT: You are using gradio version 4.16.0, however version 4.44.1 is available, please upgrade.
--------
Adjusting for ambient noise...
Please speak...
Recognizing with PocketSphinx...
Recognized Text: you spoke
